# ETL Notebook

In [50]:
import requests
import json
import pandas as pd
from datetime import datetime

# Import YT API
from secret_keys import yt_api_key
from youtube_transcript_api import YouTubeTranscriptApi

### Utility Functions

In [51]:
def getVideoInfo(response):
    """
    """
    page_items = json.loads(response.content)['items']

    video_info_list = []

    for item in page_items:
        if item['id']['kind'] == 'youtube#video':
            video_id = item['id']['videoId']
            upload_date = item['snippet']['publishedAt']
            video_title = item['snippet']['title']

            video_info_list.append({'video_id' : video_id, 'video_title' : video_title, 'upload_date' : upload_date})

    return video_info_list


In [ ]:
def getVideoTranscript(video_id : str):
    """   
    """
    try:
        # get transcript
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        transcript_text = ''
        for item in transcript:
            transcript_text += item['text'] + ' '

    except:
        # If no captions available set transcript text to "n/a"
        transcript_text = "n/a"

    return transcript_text

### Extract Process : Extract Title & Transcript Data of RWS's YT Videos

In [52]:
# Define channel ID of RWS's Youtube Channel
channel_id = "UCXYHvmfsgeS19pjIXlyCryQ"

# Define Youtube Search API URL
yt_api_url = "https://www.googleapis.com/youtube/v3/search"

# Initialize list to store video data & page token
video_info_list = []
page_token = None

In [53]:
# Extract video data from all search result pages
while page_token != 0:
    # define parameters for Youtube API request
    params = {'key' : yt_api_key, 'channelId' : channel_id,
              'part' : ["snippet", "id"], 'order' : "date",
              'maxResults' : 50, 'pageToken' : page_token}
    
    # Make API request
    response = requests.get(yt_api_url, params=params)
    video_info_list += getVideoInfo(response)

    # Make API request for next page if next page exists
    try:
        page_token = json.loads(response.text)["nextPageToken"]
    except:
        page_token = 0

In [54]:
# Store response data in a DataFrame
df = pd.DataFrame(video_info_list)
df.head()

,video_id,video_title,upload_date
0,RMPzrfXYaIo,Full Event | RWS SUPERFIGHT | 16/11/2024,2024-11-20T07:45:55Z
1,9sk2Vf9Eakg,AGE IS JUST A NUMBER! 💪🏻,2024-11-19T12:25:43Z
2,Tk4H8kZh0KI,Full Fight l Yonis Anane Venum Muay Thai vs Ja...,2024-11-17T04:30:12Z
3,4SnTIX075hk,Full Fight l Joe Ryan 2 Brothers Gym vs Petchm...,2024-11-17T04:15:00Z
4,mCpGEYufiEA,Full Fight l Khunsueklek Boomdeksian vs Kevin ...,2024-11-17T04:00:25Z


In [56]:
### Add video transcripts to Video Data dataframe
df['transcript'] = df['video_id'].apply(lambda x : getVideoTranscript(x))
df.head()

,video_id,video_title,upload_date,transcript
0,RMPzrfXYaIo,Full Event | RWS SUPERFIGHT | 16/11/2024,2024-11-20T07:45:55Z,a C good evening ladies and gentlemen we are h...
1,9sk2Vf9Eakg,AGE IS JUST A NUMBER! 💪🏻,2024-11-19T12:25:43Z,n/a
2,Tk4H8kZh0KI,Full Fight l Yonis Anane Venum Muay Thai vs Ja...,2024-11-17T04:30:12Z,this is R introducing your referee Mr [Music] ...
3,4SnTIX075hk,Full Fight l Joe Ryan 2 Brothers Gym vs Petchm...,2024-11-17T04:15:00Z,this is R introducing your referee Mr Narin po...
4,mCpGEYufiEA,Full Fight l Khunsueklek Boomdeksian vs Kevin ...,2024-11-17T04:00:25Z,this is r and introduce your referee Mr Pyon [...


### Transform Process : Clean Data & Transform Into An Appropriate Format

In [57]:
# Check dataset for duplicates
print("shape : ", df.shape)
print("# Unique Rows : ", df.shape[0]-sum(df.duplicated()))
for column in df.nunique().index:
    print(f"num unique entries in {column} column : ", df.nunique()[column])

# Remove duplicate entries
df = df.drop_duplicates(keep='first')

shape :  (514, 4)
# Unique Rows :  513
num unique entries in video_id column :  513
num unique entries in video_title column :  512
num unique entries in upload_date column :  513
num unique entries in transcript column :  205


In [58]:
# Reformat upload_date column
df['upload_date'] = df['upload_date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))

In [59]:
# Perform data cleaning
remove_list = ['[Music]', '[Applause]']
for item in remove_list:
    df['transcript'] = df['transcript'].apply(lambda x : x.replace(item, ''))

df['video_title'] = df['video_title'].apply(lambda x : x.replace(' l ', ' | '))
df['video_title'] = df['video_title'].apply(lambda x : x.replace("&#39;", "'"))

### Load Process : Load the transformed data into the project's 'data' directory

In [62]:
# Save data as parquet file
df.to_parquet('data/rws-video-transcripts.parquet')